In [147]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cross_validation import train_test_split as tts
from itertools import combinations

from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [13]:
dfRosters = pd.read_csv('rosters_cleaned.csv')
dfRosters.dropna(how = 'any', inplace = True)

In [18]:
class_mapping = {
    'SR' : 4,
    'JR' : 3,
    'SO' : 2,
    'FR' : 1
}
dfRosters['class_id'] = dfRosters['class'].apply(lambda c: class_mapping[c])
# dfRosters = dfRosters.join(pd.get_dummies(dfRosters.pos_primary))
dfRosters['conf_clean'] = dfRosters.conference.apply(lambda c: c.split(' (')[0])
dfRosters = dfRosters.join(pd.get_dummies(dfRosters.conf_clean))

In [168]:
dfRosters.ht = dfRosters.ht.map(int)

In [179]:
conferences = {c : ii for ii, c in enumerate(sorted(dfRosters.conf_clean.unique()))}
dfRosters['conference_id'] = dfRosters.conf_clean.apply(lambda c: conferences[c])

In [189]:
dfRosters['pos_id'], pos_key = pd.factorize(dfRosters.pos_primary)

In [191]:
Xcol = ['year', 'class_id', 'multipos', 'conference_id', 'pos_id']
ycol = 'ht'

In [192]:
Xtrain, Xtest, ytrain, ytest = tts(dfRosters[Xcol], dfRosters[ycol])

### Random Forest Regressor

In [133]:
def test_RF_params(Xtrain, Xtest, ytrain, ytest,
                   n_estimators = range(50, 151, 50),
                   max_features = 10,
                   max_depth = range(4, 11, 2),
                   min_samples_split = range(2, 21, 2),
                   min_samples_leaf = range(3, 10, 3)
                   ):
    
    params = []
    for n in n_estimators:
        for max_d in max_depth:
            for min_split in min_samples_split:
                for min_leaf in min_samples_leaf:
                    param = {'n_estimators' : n,
                             'max_features' : max_features,
                             'max_depth' : max_d,
                             'min_samples_split' : min_split,
                             'min_samples_leaf' : min_leaf,
                             'n_jobs' : -1
                             }
                    params.append(param)
    
    results = {'mse_test' : 100000000000}
    for p in params:
        print 'Trying: ', p
        rf_reg = RFR(**p)
        rf_reg.fit(Xtrain, ytrain)
        
        score_train = rf_reg.score(Xtrain, ytrain)
        score_test = rf_reg.score(Xtest, ytest)
        ypred_train = rf_reg.predict(Xtrain)
        ypred_test = rf_reg.predict(Xtest)
        mse_train = mean_squared_error(ytrain, ypred_train)
        mse_test = mean_squared_error(ytest, ypred_test)
        
        if mse_test < results['mse_test']:
            results = {
                    'params' : p,
                    'score_train' : score_train,
                    'score_test'  : score_test,
                    'ypred_train' : ypred_train,
                    'ypred_test'  : ypred_test,
                    'mse_train'   : mse_train,
                    'mse_test'    : mse_test,
                      }
    return results

In [130]:
best_rfr_results = test_RF_params(Xtrain, Xtest, ytrain, ytest)

Trying:  {'n_jobs': -1, 'min_samples_leaf': 3, 'n_estimators': 50, 'min_samples_split': 2, 'max_features': 10, 'max_depth': 4}
Trying:  {'n_jobs': -1, 'min_samples_leaf': 6, 'n_estimators': 50, 'min_samples_split': 2, 'max_features': 10, 'max_depth': 4}
Trying:  {'n_jobs': -1, 'min_samples_leaf': 9, 'n_estimators': 50, 'min_samples_split': 2, 'max_features': 10, 'max_depth': 4}
Trying:  {'n_jobs': -1, 'min_samples_leaf': 3, 'n_estimators': 50, 'min_samples_split': 4, 'max_features': 10, 'max_depth': 4}
Trying:  {'n_jobs': -1, 'min_samples_leaf': 6, 'n_estimators': 50, 'min_samples_split': 4, 'max_features': 10, 'max_depth': 4}
Trying:  {'n_jobs': -1, 'min_samples_leaf': 9, 'n_estimators': 50, 'min_samples_split': 4, 'max_features': 10, 'max_depth': 4}
Trying:  {'n_jobs': -1, 'min_samples_leaf': 3, 'n_estimators': 50, 'min_samples_split': 6, 'max_features': 10, 'max_depth': 4}
Trying:  {'n_jobs': -1, 'min_samples_leaf': 6, 'n_estimators': 50, 'min_samples_split': 6, 'max_features': 10, 

In [139]:
best_rfr_results

{'mse_test': 4.3783880485253812,
 'mse_train': 4.2387116667300981,
 'params': {'max_depth': 10,
  'max_features': 10,
  'min_samples_leaf': 3,
  'min_samples_split': 16,
  'n_estimators': 150,
  'n_jobs': -1},
 'score_test': 0.66495633865393988,
 'score_train': 0.66752959970865844,
 'ypred_test': array([ 82.7705006 ,  78.958754  ,  74.16930691, ...,  74.3103352 ,
         73.89038719,  74.52271352]),
 'ypred_train': array([ 74.52676149,  74.1421838 ,  79.25132259, ...,  74.03474976,
         74.14280303,  74.05413138])}

In [215]:
rfr_best = RFR(n_estimators = 100,
               # max_features = 10,
               max_depth = 10,
               min_samples_split = 2,
               min_samples_leaf = 3
               )

In [216]:
rfr_best.fit(Xtrain, ytrain)

RandomForestRegressor(bootstrap=True, compute_importances=None,
           criterion='mse', max_depth=10, max_features='auto',
           max_leaf_nodes=None, min_density=None, min_samples_leaf=3,
           min_samples_split=2, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0)

In [217]:
print rfr_best.score(Xtrain, ytrain)
print rfr_best.score(Xtest, ytest)

ypred = rfr_best.predict(Xtest)
print 'mse: ', mean_squared_error(ytest.tolist(), ypred.tolist())
print 'mae: ', mean_absolute_error(ytest.tolist(), ypred.tolist())

0.677685553133
0.665431888664
mse:  4.379360536
mae:  1.63479276898


### Linear Regressions

##### Testing Elastic, Ridge and Lasso

In [224]:
def brute_force(data, target_variable, predictors, model, alpha_list = [1.0], degree_list = [3]):
    min_mse = 1e99
    test_size_split = 0.5

    #search over every combination of the predictors - using the itertools functionality
    for i in xrange(1, len(predictors)):
        
        #build and test a model for each combination of predictors
        for j in combinations(predictors, i):
            
            test_predictors = list(j)
            
            #use train test split to get the training and test datasets, according to the parameter test_size_split
            X_train, X_test, y_train, y_test = tts(data[test_predictors], \
                                                    data[target_variable], test_size=test_size_split, random_state=42)
            
            #Now search over all the polynomial degrees in the degree_list
            for degree in degree_list:
                
                #Make sure each model is regularized, and search over all alphas in the regularization list
                for a in alpha_list:
                    
                    #build the model
                    clf = make_pipeline(PolynomialFeatures(degree), model(alpha = a))
                    
                    #fit the model
                    clf.fit(X_train, y_train)
                    
                    #Get the test set predictions
                    y_hat = clf.predict(X_test)
                    
                    #measure the mean squared error of the test set
                    mse = mean_squared_error(y_hat.tolist(), y_test.tolist())
                    
                    #remember ALL information for the minimum
                    if mse < min_mse:
                        min_mse = mse
                        min_clf = clf
                        min_predictors = test_predictors
                        min_degree = degree
                        min_alpha = a
                        #unless you cannot afford to do this, it is always a good idea to remember the train, test
                        #datasets actually used to build your model
                        min_X_train = X_train
                        min_y_train = y_train
                        min_X_test = X_test
                        min_y_test = y_test
                    
    #return a tuple for the minimum model and parameters
    return (min_mse, min_clf, min_predictors, min_degree, min_alpha, min_X_train, min_y_train, min_X_test, min_y_test)

In [230]:
ridge_bf = brute_force(data = dfRosters,
                       target_variable = ycol,
                       predictors = Xcol,
                       model = Ridge, 
                       degree_list = range(len(Xcol)),
                       alpha_list = [1, 5, 10, 100]
                      )

In [234]:
lasso_bf = brute_force(data = dfRosters,
                       target_variable = ycol,
                       predictors = Xcol,
                       model = Lasso, 
                       degree_list = range(len(Xcol)),
                       alpha_list = [.0000001, .0001, .1, 1]
                      )

In [236]:
elastic_bf = brute_force(data = dfRosters,
                       target_variable = ycol,
                       predictors = Xcol,
                       model = ElasticNet, 
                       degree_list = range(len(Xcol)),
                       alpha_list = [.1, 1, 10]
                      )

In [237]:
elastic_bf

(4.4425527196861445,
 Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=4, include_bias=True, interaction_only=False)), ('elasticnet', ElasticNet(alpha=0.1, copy_X=True, fit_intercept=True, l1_ratio=0.5,
       max_iter=1000, normalize=False, positive=False, precompute='auto',
       tol=0.0001, warm_start=False))]),
 ['year', 'multipos', 'conference_id', 'pos_id'],
 4,
 0.1,
 array([[2007.0, False, 12, 2],
        [2010.0, False, 0, 2],
        [2005.0, False, 16, 2],
        ..., 
        [2013.0, True, 35, 0],
        [2001.0, False, 18, 2],
        [2004.0, False, 14, 1]], dtype=object),
 array([67, 69, 72, ..., 76, 71, 83]),
 array([[2012.0, False, 14, 0],
        [2005.0, False, 18, 0],
        [2005.0, False, 6, 0],
        ..., 
        [2012.0, False, 3, 0],
        [2011.0, False, 31, 0],
        [2012.0, False, 10, 2]], dtype=object),
 array([80, 78, 80, ..., 77, 79, 72]))

In [235]:
lasso_bf

(4.4424479917809885,
 Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=4, include_bias=True, interaction_only=False)), ('lasso', Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
    normalize=False, positive=False, precompute='auto', tol=0.0001,
    warm_start=False))]),
 ['year', 'multipos', 'conference_id', 'pos_id'],
 4,
 0.1,
 array([[2007.0, False, 12, 2],
        [2010.0, False, 0, 2],
        [2005.0, False, 16, 2],
        ..., 
        [2013.0, True, 35, 0],
        [2001.0, False, 18, 2],
        [2004.0, False, 14, 1]], dtype=object),
 array([67, 69, 72, ..., 76, 71, 83]),
 array([[2012.0, False, 14, 0],
        [2005.0, False, 18, 0],
        [2005.0, False, 6, 0],
        ..., 
        [2012.0, False, 3, 0],
        [2011.0, False, 31, 0],
        [2012.0, False, 10, 2]], dtype=object),
 array([80, 78, 80, ..., 77, 79, 72]))

In [231]:
ridge_bf

(4.4383561426370006,
 Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=4, include_bias=True, interaction_only=False)), ('ridge', Ridge(alpha=100, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, solver='auto', tol=0.001))]),
 ['year', 'multipos', 'conference_id', 'pos_id'],
 4,
 100,
 array([[2007.0, False, 12, 2],
        [2010.0, False, 0, 2],
        [2005.0, False, 16, 2],
        ..., 
        [2013.0, True, 35, 0],
        [2001.0, False, 18, 2],
        [2004.0, False, 14, 1]], dtype=object),
 array([67, 69, 72, ..., 76, 71, 83]),
 array([[2012.0, False, 14, 0],
        [2005.0, False, 18, 0],
        [2005.0, False, 6, 0],
        ..., 
        [2012.0, False, 3, 0],
        [2011.0, False, 31, 0],
        [2012.0, False, 10, 2]], dtype=object),
 array([80, 78, 80, ..., 77, 79, 72]))